# Question 1:

In [126]:
from gurobipy import *
import gurobipy as gp

import pandas as pd

In [81]:
cards, profit = gp.multidict({
    'HyperLink' : 53,
    'FastLink'  : 48,
    'SpeedLink' : 33,
    'MicroLink' : 32,
    'EtherLink' : 38
})

devices, devices_lower_bound, devices_upper_bound = gp.multidict({
    'Circuit'  :  [0, 80000],
    'Resistors':  [0, 100000],
    'Memory'   :  [0, 30000],
    'Labor'    :  [0, 5000]
})

device_value = {
    ('HyperLink', 'Circuit'): 20,
    ('HyperLink', 'Resistors'): 28,
    ('HyperLink', 'Memory'): 8,
    ('HyperLink', 'Labor'): 0.75,
    ('FastLink', 'Circuit'): 15,
    ('FastLink', 'Resistors'): 24,
    ('FastLink', 'Memory'): 8,
    ('FastLink', 'Labor'): 0.6,
    ('SpeedLink', 'Circuit'): 10,
    ('SpeedLink', 'Resistors'): 18,
    ('SpeedLink', 'Memory'): 4,
    ('SpeedLink', 'Labor'): 0.5,
    ('MicroLink', 'Circuit'): 8,
    ('MicroLink', 'Resistors'): 12,
    ('MicroLink', 'Memory'): 4,
    ('MicroLink', 'Labor'): 0.65,
    ('EtherLink', 'Circuit'): 5,
    ('EtherLink', 'Resistors'): 16,
    ('EtherLink', 'Memory'): 6,
    ('EtherLink', 'Labor'): 1
}

In [82]:
Link = Model('Interface cards')

## Objective Function

In [83]:
number = Link.addVars(cards, name="number")

In [84]:
Link.setObjective(number.prod(profit), GRB.MAXIMIZE)

In [85]:
Link.addConstrs((gp.quicksum(device_value[c, d] * number[c] for c in cards)\
                 == [devices_lower_bound[d], devices_upper_bound[d]]\
                      for d in devices), "_")

{'Circuit': <gurobi.Constr *Awaiting Model Update*>,
 'Resistors': <gurobi.Constr *Awaiting Model Update*>,
 'Memory': <gurobi.Constr *Awaiting Model Update*>,
 'Labor': <gurobi.Constr *Awaiting Model Update*>}

In [86]:
# Add the constraint that each variable must be greater than 500
Link.addConstrs(number[c] >= 500 for c in cards)

{'HyperLink': <gurobi.Constr *Awaiting Model Update*>,
 'FastLink': <gurobi.Constr *Awaiting Model Update*>,
 'SpeedLink': <gurobi.Constr *Awaiting Model Update*>,
 'MicroLink': <gurobi.Constr *Awaiting Model Update*>,
 'EtherLink': <gurobi.Constr *Awaiting Model Update*>}

In [87]:
Link.addConstr(2 * number['HyperLink'] <= number['FastLink'])

<gurobi.Constr *Awaiting Model Update*>

In [88]:
Link.write('InterfaceCards.lp')

In [89]:
def printSolution():
    if Link.status == GRB.OPTIMAL:
        print('\nProfit: %g' % Link.objVal)
        print('\nnumber:')
        for c in cards:
            if number[c].x > 0.0001:
                print('%s %g' % (c, number[c].x))
    else:
        print('No solution')

In [90]:
# Solve
Link.optimize()
printSolution()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 9 columns and 31 nonzeros
Model fingerprint: 0xc30dd8ff
Coefficient statistics:
  Matrix range     [5e-01, 3e+01]
  Objective range  [3e+01, 5e+01]
  Bounds range     [5e+03, 1e+05]
  RHS range        [5e+02, 1e+05]
Presolve removed 5 rows and 4 columns
Presolve time: 0.00s
Presolved: 5 rows, 5 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5660000e+05   1.167975e+04   0.000000e+00      0s
       3    2.1500000e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.150000000e+05

Profit: 215000

number:
HyperLink 500
FastLink 1000
SpeedLink 1500
MicroLink 2250
EtherLink 500


## Could Valu-Com make more money if it schedules assembly workers to work overtime?

In [93]:
devices, devices_lower_bound, devices_upper_bound = gp.multidict({
    'Circuit'  :  [0, 80000],
    'Resistors':  [0, 100000],
    'Memory'   :  [0, 30000],
    'Labor'    :  [0, GRB.INFINITY]
})
Link.addConstrs((gp.quicksum(device_value[c, d] * number[c] for c in cards)\
                 == [devices_lower_bound[d], devices_upper_bound[d]]\
                      for d in devices), "_")

{'Circuit': <gurobi.Constr *Awaiting Model Update*>,
 'Resistors': <gurobi.Constr *Awaiting Model Update*>,
 'Memory': <gurobi.Constr *Awaiting Model Update*>,
 'Labor': <gurobi.Constr *Awaiting Model Update*>}

In [94]:
# Solve
Link.optimize()
printSolution()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 14 rows, 12 columns and 54 nonzeros
Coefficient statistics:
  Matrix range     [5e-01, 3e+01]
  Objective range  [3e+01, 5e+01]
  Bounds range     [5e+03, 1e+05]
  RHS range        [5e+02, 1e+05]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1500000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.150000000e+05

Profit: 215000

number:
HyperLink 500
FastLink 1000
SpeedLink 1500
MicroLink 2250
EtherLink 500


It is same as the amount without this extra constraint which is still 215000.
therefore, if it schedules assmebly workers to work overtime, it does not change the total profit.
Because if u increase # of hours the profit stays the same

# Question 2

In [151]:
supply_locs, supply = gp.multidict({
    'Location 1': 16,
    'Location 2': 18
    })

demand_locs, demand_lower, demand_upper = gp.multidict({
    'Location 3': [5,10],
    'Location 4': [5,10],
    'Location 5': [5,10],
    'Location 6': [5,10]
    })

arcs, cost = gp.multidict({
    ('Location 1', 'Location 3'): 54,
    ('Location 1', 'Location 4'): 17,
    ('Location 1', 'Location 5'): 23,
    ('Location 1', 'Location 6'): 30,
    ('Location 2', 'Location 3'): 24,
    ('Location 2', 'Location 4'): 18,
    ('Location 2', 'Location 5'): 19,
    ('Location 2', 'Location 6'): 31,
})

## Objective Function

In [152]:
car = gp.Model('Transshipment')
flow = car.addVars(arcs, obj=cost, name="flow")

In [153]:
demand_flow = car.addConstrs((gp.quicksum(flow.select("*", d_loc)) \
                             == [demand_lower[d_loc], demand_upper[d_loc]]\
                                for d_loc in demand_locs), "_") 



In [154]:
supply_flow = car.addConstrs((gp.quicksum(flow.select(s_loc, '*')) \
                               == supply[s_loc]\
                                for s_loc in supply_locs), "_") 

In [155]:
car.write('Transshipment.lp')

In [156]:
car.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6 rows, 12 columns and 20 nonzeros
Model fingerprint: 0x9bdc6bc1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 5e+01]
  Bounds range     [5e+00, 5e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 6 rows, 12 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.400000e+01   0.000000e+00      0s
       7    7.3000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.300000000e+02


In [157]:
product_flow = pd.DataFrame(columns=["From", "To", "Flow"])
for arc in arcs:
    if flow[arc].x > 1e-6:
        product_flow.loc[len(product_flow)] = {"From": arc[0], "To": arc[1], "Flow": flow[arc].x}
product_flow.index = [''] * len(product_flow)
product_flow

,From,To,Flow
,Location 1,Location 4,10.0
,Location 1,Location 5,1.0
,Location 1,Location 6,5.0
,Location 2,Location 3,9.0
,Location 2,Location 5,9.0


In [176]:
print('Total cost of Car Flow: ' + str(car.objVal))

Total cost of Car Flow: 730.0


# Question 3

In [160]:
Invest = Model('Investments')

In [163]:
alist = [1,2,3,4,5,6,7,8]
blist = [1,3,5,7]
clist = [1,4]
dlist = [1]

In [165]:
a = Invest.addVars(alist, name="A")
b = Invest.addVars(blist, name="B")
c = Invest.addVars(clist, name="C")
d = Invest.addVars(dlist, name="D")

## Objective Function

In [166]:
Invest.setObjective(a[1] + b[1] + c[1] + d[1], GRB.MINIMIZE)

In [169]:
Invest.addConstr(1.06*a[1] == a[2])
Invest.addConstr(1.06*a[2] + 1.14*b[1] == a[3] + b[3])
Invest.addConstr(1.06*a[3] + 1.18*c[1] == a[4] + c[4])
Invest.addConstr(1.06*a[4] + 1.14*b[3] == a[5] + b[5])
Invest.addConstr(1.06*a[5] == a[6] + 12000)
Invest.addConstr(1.06*a[6] + 1.14*b[5] + 1.18*c[4] == a[7] + b[7] + 14000)
Invest.addConstr(1.06*a[7] + 1.65*d[1] == a[8] + 16000)
Invest.addConstr(1.06*a[8] + 1.14*b[7] == 18000)

<gurobi.Constr *Awaiting Model Update*>

In [170]:
Invest.write('Investments.lp')

In [171]:
Invest.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 9 5900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 17 rows, 23 columns and 54 nonzeros
Model fingerprint: 0xc52044d8
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+04]
Presolve removed 17 rows and 23 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8149119e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.814911927e+04


In [172]:
for x in Invest.getVars():
    print(x.varName, x.x)

A[1] 0.0
A[2] 0.0
A[3] 0.0
A[4] 0.0
A[5] 0.0
A[6] 0.0
A[7] 0.0
A[8] 0.0
A[1] 0.0
A[2] 0.0
A[3] 0.0
A[4] 0.0
A[5] 11320.754716981131
A[6] 0.0
A[7] 0.0
A[8] 16981.132075471698
B[1] 18160.554379322177
B[3] 20703.03199242728
B[5] 12280.701754385966
B[7] 0.0
C[1] 0.0
C[4] 0.0
D[1] 19988.564894225274


In [173]:
print('Cash Flow: ' + str(Invest.objVal))

Cash Flow: 38149.11927354745
